In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os

## This Week's Data

In [2]:
year = 2023
week = 14

In [3]:
def _clean_name_df(df):
    df = df.copy(deep=True)
    df['h/a'] = np.where(df[1].str.contains('@'), 'Away', 'Home')
    df[['name', 'xtra']] = df[1].str.split("•", n=1, expand=True)
    df['Name'] = df['name'].str.rstrip(' F/OT')   # strip Final score designation
    df['Name'] = df['Name'].str.rstrip(' IRPQDO')  # strip injury designation
    return df[['Name','h/a']]

In [ ]:
# %pip install selenium
# download chrome driver from: https://chromedriver.chromium.org/home
# unzip chromedriver.zip

# mac install notes (I don't remember if this is what I actually did but I had these comments in my notebook ¯\_(ツ)_/¯)
# $ mv <Path>/chromedriver /usr/local/bin/.
# $ xattr -d com.apple.quarantine /usr/local/bin/chromedriver 

In [27]:
# %pip install html5lib #lxml   # If you see errors about needing these libraries

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException
import time
from io import StringIO

In [5]:
def get_week_df(offense=True, wait=3):
    position = 1 if offense else 7 #DST
    
    # Use Python's implied line continuation inside parentheses, brackets and braces for multiline fstrings
    url = (f"https://fantasydata.com/nfl/fantasy-football-leaders?position={position}"
          f"&season={year}&seasontype=1&scope=2&subscope=1&scoringsystem=4"
          f"&startweek={week}&endweek={week}&aggregatescope=1&range=3")

    driver = webdriver.Chrome()
    driver.get(url)

    if offense:
        elem = driver.find_element(By.CLASS_NAME, "show-player-id")
        elem.find_element(By.LINK_TEXT, '300').click()
    
        time.sleep(wait)  # this is a hack to wait for the table to load -> adjust to higher value if consistently getting errors
    
    name_el = driver.find_element(By.CLASS_NAME, "k-grid-content-locked")
    name = pd.read_html(StringIO(name_el.get_attribute('outerHTML')))[0]
    name = _clean_name_df(name)
    
    content_el = driver.find_element(By.CLASS_NAME, "k-grid-content")
    content = pd.read_html(StringIO(content_el.get_attribute('outerHTML')))[0]
    content.columns = ["Team", "Pos", "Week", "Opponent", "Pass_yds", "Pass_tds", "Ints", "Rush_yds", 
                       "Rush_tds", "Rec", "Rec_yds", "Rec_tds", "Fumbles", "Fpts/gm", "Fpts"]

    driver.close()
    return pd.concat([name, content], axis=1)

In [8]:
off = get_week_df(wait=5)

In [9]:
dst = get_week_df(offense=False)

In [10]:
week_df = pd.concat([off,dst])
week_df.columns = week_df.columns.str.lower()
week_df

,name,h/a,team,pos,week,opponent,pass_yds,pass_tds,ints,rush_yds,rush_tds,rec,rec_yds,rec_tds,fumbles,fpts/gm,fpts
0,Deebo Samuel,Home,SF,WR,14,SEA,0,0,0,1,1,7,149,1,0,37.0,37.00
1,Lamar Jackson,Home,BAL,QB,14,LAR,316,3,1,70,0,0,0,0,0,35.6,35.64
2,Evan Engram,Away,JAX,TE,14,CLE,0,0,0,0,0,11,95,2,0,32.5,32.50
3,Drake London,Home,ATL,WR,14,TB,0,0,0,0,0,10,172,0,0,32.2,32.20
4,Desmond Ridder,Home,ATL,QB,14,TB,347,1,1,15,1,0,0,0,0,29.4,29.38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Detroit Lions,Away,DET,DST,14,CHI,5,3,8,0,0,0,0,0,28,4.0,4.00
26,Los Angeles Chargers,Home,LAC,DST,14,DEN,3,2,5,1,0,0,0,0,24,4.0,4.00
27,Carolina Panthers,Away,CAR,DST,14,NO,4,1,6,1,0,0,0,0,22,3.0,3.00
28,Los Angeles Rams,Away,LAR,DST,14,BAL,2,2,5,1,0,1,0,0,37,2.0,2.00


In [ ]:
# week_df['type'] = week_df['fpts'].apply(lambda x: type(x))
week_df['fpts'] = pd.to_numeric(week_df['fpts'], errors='coerce')

## Check scores

In [11]:
name_to_fullname = {
    'Lions': 'Detroit Lions',
    'Dolphins': 'Miami Dolphins',
    'Chiefs': 'Kansas City Chiefs',
    'Browns': 'Cleveland Browns',
    'Buccaneers': 'Tampa Bay Buccaneers',
    'Bills': 'Buffalo Bills',
    'Patriots': 'New England Patriots',
    'Vikings': 'Minnesota Vikings',
    'Eagles': 'Philadelphia Eagles',
    'Falcons': 'Atlanta Falcons',
    'Ravens': 'Baltimore Ravens',
    'Texans': 'Houston Texans',
    'Bears': 'Chicago Bears',
    '49ers': 'San Francisco 49ers',
    'Jets': 'New York Jets',
    'Bengals': 'Cincinnati Bengals',
    'Cowboys': 'Dallas Cowboys',
    'Titans': 'Tennessee Titans',
    'Cardinals': 'Arizona Cardinals',
    'Saints': 'New Orleans Saints',
    'Raiders': 'Las Vegas Raiders',
    'Commanders': 'Washington Commanders',
    'Seahawks': 'Seattle Seahawks',
    'Colts': 'Indianapolis Colts',
    'Chargers': 'Los Angeles Chargers',
    'Rams': 'Los Angeles Rams',
    'Jaguars': 'Jacksonville Jaguars',
    'Giants': 'New York Giants',
    'Panthers': 'Carolina Panthers',
    'Broncos': 'Denver Broncos',
    'Packers': 'Green Bay Packers', 
    'Steelers': 'Pittsburgh Steelers',
}
city_to_fullname = {' '.join(n.split(' ')[:-1]): n for n in name_to_fullname.values()}

defense_map = name_to_fullname | city_to_fullname

def apply_defense_map(x):
    try:
        return defense_map[x]
    except KeyError:
        return x

In [66]:
# %pip install openpyxl   #need this to read xlsx formatted submissions

In [12]:
def change_last_name_first(df):
    ndf = df.copy()
    s = ndf[ndf.pos!='Def']['name']
    s = s.str.split(',', expand=True).fillna('')
    s['join_name'] = s[1] + " " + s[0]
    ndf.loc[ndf.pos!='Def', 'name'] = s.join_name.str.strip()
    ndf['name'] = ndf['name'].str.replace(',', '')
    return ndf

In [13]:
def standardize_submissions(df):
    "takes in submissions of all different types and standardizes them to be scored automatically"
    df.columns = map(str.lower, df.columns)  # lowercase column names
    df = df.head(9)   # only keep player rows - first 9
    # rename position to pos if it exists in columns
    if 'position' in df.columns: 
        df = df.rename({'position':'pos'}, axis=1)
    df = df.loc[:,~df.columns.duplicated()]   #remove duplicated columns if any
    df.loc[df.pos.isin(['D','Def']),'pos'] = 'DST'  # default to using DST as defense
    df = df.rename({'player':'name'}, axis=1)  # rename player to name
    df['name'] = df['name'].str.strip()   # strip whitespace from name column
    df.loc[df['pos']=='DST', 'name'] = df['name'].apply(lambda x: apply_defense_map(x))  # map defense names
    # check if name contains a comma -> convert from last, first to first last in prep for the join
    if ',' in df.loc[df['pos']=='QB', 'name'].item():
        df = change_last_name_first(df)
    return df

In [14]:
def score_submission(df, score_df=week_df):
    df = df.join(score_df[['name', 'fpts']].set_index('name'), on='name')  # join in scores dataframe
    df = df.fillna(0)  # fill empties with 0
    print(f"Total Points in Week {week}:", sum(df['fpts']))
    return df

In [15]:
def score_all_in_dir(dir):
    for f in os.listdir(dir):
        # if not (f.startswith("Sebastian") or f.startswith("Rashod")):
        # if not (f.startswith('SCORED_') or f.startswith('all_players_') or f.startswith('Week6_Totals')) :
            try:
                j = pd.read_csv(pick_dir+f)
            except UnicodeDecodeError:
                j = pd.read_excel(pick_dir+f)
            print(f)
            try:
                df = standardize_submissions(j)
                df = score_submission(df, week_df)
                print(df, end='\n\n')
            except Exception as e:
                print(e)

In [16]:
# add shortcut to Weekly Team Picks folder to OneDrive and then access from that filepath
# change this to your own filepath
pick_dir = f"/Users/adamschiller/OneDrive - BOOZ ALLEN HAMILTON/Weekly Team Picks/Week {format(week, '02d')}/"

In [17]:
score_all_in_dir(pick_dir)

Rashod_week_14.xlsx
Total Points in Week 14: 72.58
                  name  pos  salary   fpts
0      Atlanta_Falcons  DST    3100   0.00
1          Jalen Hurts   QB    8200   9.88
2  Christian McCaffrey   RB    9200  19.30
3     Javonte Willoams   RB    5800   0.00
4        Jake Ferguson   TE    4400  12.20
5          CeeDee Lamb   WR    9000  19.10
6       Devante Parker   WR    3100   0.00
7       Jonathan Mingo   WR    3500   4.20
8            Zay Jones   WR    3700   7.90

jillian_ff_week14.csv
Total Points in Week 14: 148.42000000000002
   unnamed: 0  dk_salary                 name  pos   fpts
0         0.0     2900.0      Atlanta Falcons  DST   1.00
1         1.0     6800.0        Justin Fields   QB  24.72
2         2.0     9200.0  Christian McCaffrey   RB  19.30
3         3.0     6100.0            Joe Mixon   RB  21.50
4         4.0     7200.0       Saquon Barkley   RB  24.10
5         5.0     9000.0          CeeDee Lamb   WR  19.10
6         6.0     4600.0         Drake London 

# Rectify individual DFs

In [18]:
week_df[week_df.name.str.contains('Robinson')]

,name,h/a,team,pos,week,opponent,pass_yds,pass_tds,ints,rush_yds,rush_tds,rec,rec_yds,rec_tds,fumbles,fpts/gm,fpts
30,Brian Robinson Jr.,Home,WAS,RB,11,NYG,0,0,0,73,0,7,58,0,0,20.1,20.1
109,Dorian Thompson-Robinson,Home,CLE,QB,11,PIT,165,0,1,20,0,0,0,0,0,7.6,7.6
125,Wan'Dale Robinson,Away,NYG,WR,11,WAS,0,0,0,2,0,3,35,0,0,6.7,6.7
150,Allen Robinson,Away,PIT,WR,11,CLE,0,0,0,0,0,3,20,0,0,5.0,5.0
156,Demarcus Robinson,Home,LAR,WR,11,SEA,0,0,0,0,0,2,24,0,0,4.4,4.4


In [17]:
df = pd.read_excel(pick_dir+f"Richard_week{week}_picks.xlsx")
df

,Player,Pos,FD SALARY
0,Kyler Murray,QB,6900
1,Travis Etienne,RB,7100
2,Josh Jacobs,RB,6700
3,BUF,DST,2700
4,Isiah Pacheco,RB,6200
5,Courtland Sutton,WR,5400
6,Christian Kirk,WR,5700
7,Dalton Schultz,TE,5400
8,Wan'Dale Robinson,WR,3800


In [18]:
# df.loc[4, 'Player'] = 'DJ Moore'
# df.loc[2, 'Player'] = 'Travis Etienne Jr.'
# df.loc[3, 'Player'] = 'Pittsburgh Steelers'
# df.loc[5, 'Player'] = 'Brian Robinson Jr.'
df = standardize_submissions(df)
df = score_submission(df, week_df)
print(df, end='\n\n')

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [27]:
df = pd.read_csv(pick_dir+f"SebastianCoombs_linupweek{week}.csv")
df

,pos,player,points,salary,team,player_id,draftable_id,opponent,first,last,player_name
0,QB,matthew-stafford,17.856330,6600.0,LAR,matthew-stafford,30531809,DAL,Matthew,Stafford,Matthew Stafford
1,RB,saquon-barkley,21.806370,8000.0,NYG,saquon-barkley,30532053,NYJ,Saquon,Barkley,Saquon Barkley
2,RB,jamaal-williams,11.623615,4300.0,NO,jamaal-williams,30531878,IND,Jamaal,Williams,Jamaal Williams
3,WR,devonta-smith,20.721642,6700.0,PHI,devonta-smith,30532258,WAS,DeVonta,Smith,DeVonta Smith
4,WR,chris-olave,21.366478,6400.0,NO,chris-olave,30532386,IND,Chris,Olave,Chris Olave
5,WR,jahan-dotson,13.267340,4500.0,WAS,jahan-dotson,30532382,PHI,Jahan,Dotson,Jahan Dotson
6,TE,pat-freiermuth,9.968270,2500.0,PIT,pat-freiermuth,30532365,JAX,Pat,Freiermuth,Pat Freiermuth
7,WR,cooper-kupp,28.658100,9000.0,LAR,cooper-kupp,30531880,DAL,Cooper,Kupp,Cooper Kupp
8,DST,ne,3.770027,2000.0,NE,ne,30531792,MIA,Patriots,NaN,Patriots


In [28]:
df = df.rename({'player_name':'name'}, axis=1)[['pos','name','salary']]
df = standardize_submissions(df)
df = score_submission(df, week_df)
print(df, end='\n\n')

Total Points in Week 8: 107.37999999999998
   pos                  name  salary   fpts
0   QB      Matthew Stafford  6600.0  12.38
1   RB        Saquon Barkley  8000.0  18.80
2   RB       Jamaal Williams  4300.0   4.80
3   WR         DeVonta Smith  6700.0  22.90
4   WR           Chris Olave  6400.0   9.60
5   WR          Jahan Dotson  4500.0  27.80
6   TE        Pat Freiermuth  2500.0   0.00
7   WR           Cooper Kupp  9000.0   6.10
8  DST  New England Patriots  2000.0   5.00



In [16]:
df = pd.read_excel(pick_dir+f"Rashod_week{week}.xlsx")
df

,players,pos,Salary
0,Will Levis,QB,5300
1,Jonathan Taylor,RB,6600
2,Rachaad White,RB,5800
3,Even Engram,TE,4500
4,George Pickens,WR,5100
5,Ja'Marr Chase,WR,8600
6,Marquise Brown,WR,5200
7,Michael Thomas,WR,4500
8,Dallas Cowboys,DST,4400


In [17]:
df = df.rename({'players':'name'}, axis=1)
df.loc[3, 'name'] = "Evan Engram"
# df.loc[6, 'Player'] = "Jaxon Smith-Njigba"
df = standardize_submissions(df)
df = score_submission(df, week_df)
print(df, end='\n\n')

Total Points in Week 10: 92.06000000000002
              name  pos  salary   fpts
0       Will Levis   QB    5300   6.96
1  Jonathan Taylor   RB    6600  14.50
2    Rachaad White   RB    5800  17.80
3      Evan Engram   TE    4500   5.20
4   George Pickens   WR    5100   7.50
5    Ja'Marr Chase   WR    8600  26.40
6   Marquise Brown   WR    5200   3.80
7   Michael Thomas   WR    4500   1.90
8   Dallas Cowboys  DST    4400   8.00

